In [1]:
#importing libs
import urllib.request
import pandas as pd
import glob
import os
from datetime import datetime


In [2]:
directory = 'csv_lab2'
if(not os.path.exists(directory)):
        print("Create dir...")  
        os.makedirs(directory)
else:
        print("Dir already exists.")  
    
if(len(os.listdir(directory))):
        print("Files already exist.")  
else:
    print("Download files...")  
    for i in range(1, 29):
        print("Downloading for ID:{}".format(i))   
        url = 'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2020&type=Mean'.format(i)  # Generate the URL for downloading the file
        wp = urllib.request.urlopen(url)
        text = wp.read()
        now_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        out = open(directory+"/"+f'NOAA_ID_{i}_{now_time}.csv','wb')
        out.write(text)
        out.close()

Create dir...
Download files...


In [3]:
files = os.listdir(directory)
df_all = pd.DataFrame()

for file in files:
    headers = ['Year', 'Week', ' SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    df = pd.read_csv(f'{directory}/{file}', header=1, names=headers, skiprows=1)[:-1]
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df['area'] = int(file.split('_')[2])
    df = df.drop(columns=['empty'])
    df_all = df_all.dropna(axis=1, how='all')
    df_all = pd.concat([df_all, df], ignore_index=True)

print(df_all, "\n")

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04    10
1      1982   3.0  0.063  263.45  57.30  32.69  44.99    10
2      1982   4.0  0.061  265.10  53.96  28.62  41.29    10
3      1982   5.0  0.058  266.42  46.87  28.57  37.72    10
4      1982   6.0  0.056  267.47  39.55  30.27  34.91    10
...     ...   ...    ...     ...    ...    ...    ...   ...
55351  2020  48.0  0.159  277.21  66.52  16.25  41.37     9
55352  2020  49.0  0.150  275.06  65.68  19.44  42.55     9
55353  2020  50.0  0.144  274.00  66.17  18.69  42.43     9
55354  2020  51.0  0.138  272.73  66.79  19.59  43.19     9
55355  2020  52.0  0.132  271.44  67.24  21.90  44.57     9

[55356 rows x 8 columns] 



In [4]:
#change indexes
indexes = ["22", "24", "23", "25", "3", "4", "8", "19", "20", "21", "9", "26", "10", "11",
                            "12", "13", "14", "15", "16", "27", "17", "18", "6", "1", "2", "7", "5"]  
old = 1
for new in indexes:
    df_all.area.replace({old: new}, inplace=True)  
    old += 1
    df_all.to_csv(f"{directory}/ALL_CSV.csv")  
print(df_all)
df_all['Year'] = df_all["Year"].astype(int)
df_all['area'] = df_all["area"].astype(int)

C:\Users\ethan\AppData\Local\Temp\ipykernel_27244\1685136713.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_all.area.replace({old: new}, inplace=True)


       Year  Week    SMN     SMT    VCI    TCI    VHI area
0      1982   2.0  0.063  261.53  55.89  38.20  47.04   21
1      1982   3.0  0.063  263.45  57.30  32.69  44.99   21
2      1982   4.0  0.061  265.10  53.96  28.62  41.29   21
3      1982   5.0  0.058  266.42  46.87  28.57  37.72   21
4      1982   6.0  0.056  267.47  39.55  30.27  34.91   21
...     ...   ...    ...     ...    ...    ...    ...  ...
55351  2020  48.0  0.159  277.21  66.52  16.25  41.37   20
55352  2020  49.0  0.150  275.06  65.68  19.44  42.55   20
55353  2020  50.0  0.144  274.00  66.17  18.69  42.43   20
55354  2020  51.0  0.138  272.73  66.79  19.59  43.19   20
55355  2020  52.0  0.132  271.44  67.24  21.90  44.57   20

[55356 rows x 8 columns]


In [5]:
# VHI for year and min|max
def find_vhi(area, year):
    
    df_search = df_all[(df_all.area == area) & (df_all.Year == year)]['VHI']
    print(f"Here are VHI for province {area} in {year}")  
    print(df_search)  
    print("...")

def find_extremums(area, year):
    max_v = df_all[(df_all.Year.astype(int) == int(year)) & (df_all.area == area)]['VHI'].max()  
    min_v = df_all[(df_all.Year.astype(int) == int(year)) & (df_all.area == area)]['VHI'].min()  
    print(f'The MAX value is: {max_v}')  
    print(f'The MIN value is: {min_v}')  
    print("...")
    return


In [6]:
find_vhi(6, 1999)

Here are VHI for province 6 in 1999
28531    35.16
28532    38.50
28533    43.96
28534    48.92
28535    51.29
28536    53.40
28537    54.47
28538    54.82
28539    54.11
28540    51.83
28541    50.92
28542    49.08
28543    47.36
28544    46.04
28545    45.47
28546    48.51
28547    53.94
28548    59.28
28549    59.55
28550    57.94
28551    56.88
28552    56.72
28553    54.40
28554    51.42
28555    49.65
28556    48.88
28557    50.75
28558    52.65
28559    55.31
28560    58.89
28561    62.14
28562    64.87
28563    65.57
28564    67.76
28565    68.54
28566    63.39
28567    57.69
28568    51.90
28569    47.76
28570    48.36
28571    48.62
28572    46.80
28573    46.38
28574    43.26
28575    38.49
28576    35.36
28577    34.77
28578    35.00
28579    34.72
28580    30.75
28581    30.26
28582    31.75
Name: VHI, dtype: float64
...


In [7]:
find_extremums(6, 1999)

The MAX value is: 68.54
The MIN value is: 30.26
...


In [8]:
def vhi_range(areas, min_year, max_year):
    if not isinstance(areas, list) or not areas:
        return print('No data')
    return df_all[(df_all['Year'] >= min_year) & (df_all['Year'] <= max_year) & (df_all['area'].isin(areas))][['Year', 'VHI', 'area']]

In [9]:
vhi_range([2, 3], 1998, 2003)

,Year,VHI,area
32433,1998,47.28,2
32434,1998,49.05,2
32435,1998,50.03,2
32436,1998,49.41,2
32437,1998,49.07,2
...,...,...,...
46578,2003,46.43,3
46579,2003,52.15,3
46580,2003,57.91,3
46581,2003,65.84,3


In [10]:
#droughts more than given percent
def extreme_droughts(prc):
    df_drought = df_all[(df_all['VHI'] <= 15) & (df_all['VHI'] != -1)]
    group = df_drought.groupby('Year')['area'].nunique()
    res = group[group > ((25*prc)/100)].reset_index()
    return res

In [11]:
extreme_droughts(20)

,Year,area
0,2000,6


In [12]:
def mid_droughts(prc, min=15, max=40):
    df_drought = df_all[(df_all['VHI'] >= min) & (df_all['VHI'] <= max)]
    group = df_drought.groupby('Year')['area'].nunique()
    areas = df_all['area'].nunique()
    res = group[group > ((areas* prc)/100)].reset_index()
    return res

In [13]:
mid_droughts(20)

,Year,area
0,1982,28
1,1983,27
2,1984,28
3,1985,28
4,1986,28
5,1987,28
6,1988,28
7,1989,28
8,1990,28
9,1991,27
